In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/spaceship-titanic/train.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv("/kaggle/input/d/gopikrishnayogan/spaceship-titanic/train.csv")
train_df.head()

In [ ]:
train_df.info()

1. **PassengerId** - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
2. **HomePlanet** - The planet the passenger departed from, typically their planet of permanent residence.
3. **CryoSleep** - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
4. **Cabin** - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
5. **Destination** - The planet the passenger will be debarking to.
6. **Age** - The age of the passenger.
7. **VIP** - Whether the passenger has paid for special VIP service during the voyage.
8. **RoomService, FoodCourt, ShoppingMall, Spa, VRDeck** - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
9. **Name** - The first and last names of the passenger.
10. **Transported**- Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

- There are 8693 rows and 13 columns are there
- our Target variable is boolean type
- It seems to be there is some null values in some of the columns

In [ ]:
train_df.duplicated().sum()

- There is no Dublicates are present 

In [ ]:
train_df.isna().sum()

* There is so many Null values in the features
* Null values can be imputed or dropped

In [ ]:
train_df.describe().T

- Most of the Numerical columns are seems to be zero mean Except Age column

In [ ]:
train_df.describe(include='object').T

## Observations :-
- PassengerId is unique feature
- HomePlanet having 3 unique values and major are from earth
- CryoSleep - Most of the passengers are not in hybernation or sleep state
- Cabin - seems to be suspicious we may divide this lets check once
- Most of the Passengers are not VIP 
- There are ony three Destinations are there
- It seems to be Name is also unique columns,needed to check

In [ ]:
train_df[['cabin_deck','cabin_num','cabin_side']] = train_df['Cabin'].str.split('/',expand=True)

In [ ]:
train_df.sample(2)

In [ ]:
train_df.describe(include='object').T

- Now its some what clear that cabindeck divided into three columns

In [ ]:
train_df.columns

In [ ]:
train_df.drop(['PassengerId','Cabin'],axis=1,inplace=True)

In [ ]:
cat_feat = [feat for feat in train_df.columns if train_df[feat].nunique() <= 25]
num_feat = [feat for feat in train_df.columns if train_df[feat].nunique() >= 25]
print("Categorical features : \n", cat_feat)
print("Numerical features : \n", num_feat)

In [ ]:
train_df['Transported'].value_counts().plot(kind='bar');

- Classes are balanced

In [ ]:
transported = train_df['Transported'].astype('str')
print(transported)

In [ ]:
train_cat = train_df[cat_feat]
train_cat.head()

In [ ]:
[train_cat[x].value_counts() for x in train_cat.columns]

In [ ]:
train_cat.isna().mean()

In [ ]:
def null_target_relation(catfeat,c='darkblue'):
    relation = train_cat[train_cat[catfeat].isnull()]['Transported'].value_counts()
    relation.plot(kind='bar',color=c)
    plt.title(str(catfeat)+' and Transported')
    return pd.DataFrame(relation)

In [ ]:
null_target_relation('HomePlanet')

In [ ]:
null_target_relation('CryoSleep',c='green')

In [ ]:
null_target_relation('Destination',c='blue')

In [ ]:
null_target_relation('VIP',c='brown')

In [ ]:
null_target_relation('cabin_deck',c='navy')

In [ ]:
null_target_relation('cabin_side',c='orange')

- Null values are equally impact on both categories of our Target feature

In [ ]:
train_cat.groupby('HomePlanet')['CryoSleep'].value_counts()

In [ ]:
train_cat.groupby('HomePlanet')['Destination'].value_counts()

In [ ]:
train_cat.groupby('HomePlanet')['VIP'].value_counts()

* More number of VIP passengers are from Europa and Mars
* No VIP Passenger from Earth 

In [ ]:
train_cat.groupby('HomePlanet')['cabin_deck'].value_counts()

In [ ]:
train_cat.groupby('HomePlanet')['cabin_side'].value_counts()

In [ ]:
train_cat.groupby('CryoSleep')['VIP'].value_counts()

In [ ]:
train_num = train_df[num_feat]
train_num.head()

In [ ]:
train_df[train_df['Age'].isna()]['Transported'].value_counts()

In [ ]:
train_df[train_df['RoomService'].isna()]['Transported'].value_counts()

- Numerical columns are also equally impacts on Target feature

In [ ]:
train_df[train_df['Age']==0.0]['Age'].value_counts()

- The Age never be zero or children cannot do this type of travel


In [ ]:
train_df['Age'].isna().sum()

In [ ]:
train_df['Age'].plot(kind='hist');

In [ ]:
sns.boxplot(train_df['Age'])

In [ ]:
train_df.head()

In [ ]:
ax = plt.figure(figsize=(20,15))
train_num.hist(ax=ax,color='darkblue',grid=False);

In [ ]:
vip_roomservice = train_df[train_df['VIP']==False]['RoomService'] #['FoodCourt','ShoppingMall','Spa','VRDeck']]

In [ ]:
vip_roomservice.min(),vip_roomservice.max()

In [ ]:
vip_roomservice = train_df[train_df['VIP']==True]['RoomService'] #['FoodCourt','ShoppingMall','Spa','VRDeck']]

In [ ]:
vip_roomservice.hist()

In [ ]:
vip_roomservice.min(),vip_roomservice.max()

- There is some issue with RoomService feature ,The passengers who dont have VIP ,charges are more

In [ ]:
vip_fc = train_df[train_df['VIP']==True]['FoodCourt'] #['FoodCourt','ShoppingMall','Spa','VRDeck']]

In [ ]:
sns.distplot(vip_fc)
vip_fc.value_counts().head(10)

In [ ]:
vip_fc = train_df[train_df['VIP']==False]['FoodCourt'] #['FoodCourt','ShoppingMall','Spa','VRDeck']]
sns.distplot(vip_fc);

In [ ]:
vip_fc.value_counts().head()

* its been clear that vip food charges are more for Vip Passngers

In [ ]:
train_df.loc[train_df['VIP']==False,['ShoppingMall','Spa']].hist(grid=False,color='magenta') #['FoodCourt','ShoppingMall','Spa','VRDeck']]


In [ ]:
train_df.loc[train_df['VIP']==True,['ShoppingMall','Spa']].hist(grid=False,color='green') #['FoodCourt','ShoppingMall','Spa','VRDeck']]


In [ ]:
#lets check correlation plot
plt.figure(figsize=(10,10))
sns.heatmap(train_num.drop('Name',axis=1).corr(),annot=True,cbar=False)
plt.xticks(rotation=90)
plt.yticks(rotation=0);

In [ ]:
sum(train_num['Name'].value_counts()==2)

In [ ]:
train_df1 = train_df.drop('Name',axis=1)
train_df1['Transported'] = train_df1['Transported'].astype('object')
train_df1['Transported'] = train_df1['Transported'].map({
    True : 1,
    False :0
})

In [ ]:
train_df1.head()

In [ ]:
train_df1.info()

In [ ]:
train_df2 = train_df1.select_dtypes([int,float])
train_df2.head()

In [ ]:
sns.pairplot(train_df2,hue='Transported')

### Preprocessing

In [ ]:
train_df1.shape

In [ ]:
train_df_001 = train_df1.copy()

* Dropping Null values

In [ ]:
train_df1.dropna(inplace = True)

In [ ]:
train_df_fin= train_df1.reset_index()

In [ ]:
X = train_df_fin.drop('Transported',axis=1)
y = train_df_fin['Transported']

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
X_train.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for clmn in X_train.columns:
    X_train[clmn] = le.fit_transform(X_train[clmn])
    X_test[clmn]  = le.fit_transform(X_test[clmn])

In [ ]:
X_test.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

In [ ]:
rf.score(X_test,y_test)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

### Impute null values

In [ ]:
train_df_001.head()

In [ ]:
train_df_001.isna().sum()

In [ ]:
train_df_001.info()

In [ ]:
train_df_001['cabin_num'] = train_df_001['cabin_num'].astype(float)

In [ ]:
cat_clmns = train_df_001.select_dtypes(include=object)
for col in cat_clmns.columns:
    cat_clmns[col] = cat_clmns[col].fillna(cat_clmns[col].mode()[0])

In [ ]:
cat_clmns.isna().sum()

In [ ]:
le1 = LabelEncoder()
for col in cat_clmns.columns:
    le1.fit(cat_clmns[col])
    cat_clmns[col] = le1.transform(cat_clmns[col])

In [ ]:
cat_clmns.head()

In [ ]:
num_clmns = train_df_001.select_dtypes(exclude=[object])
for col in num_clmns.columns:
    num_clmns[col] = num_clmns[col].fillna(num_clmns[col].median())

final_df = pd.concat([cat_clmns,num_clmns],axis=1)
final_df.head()

In [ ]:
final_df.info()

In [ ]:
X = final_df.drop('Transported',axis=1)
y = final_df['Transported']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_1 = scaler.transform(X_train)
X_test_1 = scaler.transform(X_test)

In [ ]:
minmax = MinMaxScaler()
minmax.fit(X_train)
X_train_2  = minmax.transform(X_train)
X_test_2 = minmax.transform(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,roc_auc_score

In [ ]:
def model_evaluate(model,X_tr,y_tr,X_ts,y_ts):
    print("*-*"*40)
    print(model)
    model.fit(X_tr,y_tr)
    y_pred = model.predict(X_ts)
    print("Training score : ",model.score(X_tr,y_tr))
    print("*"*30)
    print("Testing score " , model.score(X_ts,y_ts))
    print("*"*30)
    print("classification report : \n ",classification_report(y_ts,y_pred))
    print("*"*30)
    cm = confusion_matrix(y_ts,y_pred)
    print(cm)
    print("roc_auc score : \n ",roc_auc_score(y_ts,y_pred))
    
    

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

lgt = LogisticRegression()
nb = GaussianNB()
svm = SVC()
dtree = DecisionTreeClassifier()
knn = KNeighborsClassifier()

models = [lgt,nb,svm,dtree,knn]

In [ ]:
for model in models:
    model_evaluate(model,X_train_1,y_train,X_test_1,y_test)

* SVM,KNN and LogisticRegression performs quite better

In [ ]:
for model in models:
    model_evaluate(model,X_train_2,y_train,X_test_2,y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier,StackingClassifier
rf = RandomForestClassifier()
abc = AdaBoostClassifier()
bgc = BaggingClassifier()
gbc = GradientBoostingClassifier()

ensemble = [rf,abc,abc,gbc]

for model in ensemble:
    model_evaluate(model,X_train_1,y_train,X_test_1,y_test)

### Hyperparameter Tuning

In [ ]:
scaler_1 = StandardScaler()
scaler_1.fit(X)
X_1 = scaler_1.transform(X)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the SVM classifier
svm_classifier = SVC()

# Define a random search grid of hyperparameters
param_dist = {
    'C': [0.1, 1, 2,5,10],              # Regularization parameter
    'kernel': ['linear', 'rbf','sigmoid','poly'],    # Kernel function
    'gamma': [0.1, 1, 'scale']      # Kernel coefficient (only for 'rbf' kernel)
}

# Create a random search object
random_search = RandomizedSearchCV(
    svm_classifier, param_distributions=param_dist, n_iter=10, cv=5, random_state=42
)

# Perform random search to find the best hyperparameters
random_search.fit(X_1, y)

# Get the best hyperparameters and the corresponding model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)


In [ ]:
best_model.fit(X_train_1,y_train)

In [ ]:
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test_df.head()

In [ ]:
id = test_df['PassengerId']

In [ ]:
test_df.drop(['PassengerId','Name'],axis=1,inplace=True)

In [ ]:
test_df[['cabin_deck','cabin_num','cabin_side']] = test_df['Cabin'].str.split('/',expand=True)
test_df.drop('Cabin',axis=1,inplace=True)
test_df.head()

In [ ]:
test_df['cabin_num'] = test_df['cabin_num'].astype(float)
cat_clmns = test_df.select_dtypes(include=object)
for col in cat_clmns.columns:
    cat_clmns[col] = cat_clmns[col].fillna(cat_clmns[col].mode()[0])
for col in cat_clmns.columns:
    cat_clmns[col] = le1.fit_transform(cat_clmns[col])
    
num_clmns = test_df.select_dtypes(exclude=[object])
for col in num_clmns.columns:
    num_clmns[col] = num_clmns[col].fillna(num_clmns[col].median())

final_test = pd.concat([cat_clmns,num_clmns],axis=1)
final_test.head()

In [ ]:
final_test_1 = scaler.transform(final_test)
final_test_1

In [ ]:
predicted = best_model.predict(final_test_1)

In [ ]:
pd.DataFrame(predicted)

In [ ]:
submission_val = pd.DataFrame(predicted)[0].map({
    1 : True,
    0 : False
})

In [ ]:
submission = pd.DataFrame({ 'PassengerId' : id,
                           'Transported' : submission_val
    
})

submission.head()

In [ ]:
submission.to_csv('Submission_1.csv',index=False)